# Zillow Notebook

### James Allen


## Notebook Agenda

1. Exective Summary
2. Import Zillow Data
3. Wrangle Zillow Data
4. Data Exploration and Visuals
5. Data Modeling
6. Conclusion / Next Steps

# 1. Executive Summary

- This is where I will write my executive summary 
- goals
- takeaways

In [1]:
# imports

import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoLars
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler
import sklearn.preprocessing
import scipy.stats as stats

from env import host, user, password
import acquire
import wrangle_zillow

# 2. Import the Zillow Data

In [2]:
# getting zillow data from acquire file
df = acquire.get_zillow_data()

In [3]:
df.head() #check_yo_head

,parcelid,logerror,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,propertylandusetypeid,regionidcity,regionidcounty,regionidzip,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount
0,14297519,0.025595,3.5,4.0,3100.0,6059.0,33634931.0,-117869207.0,4506.0,261.0,53571.0,1286.0,96978.0,1998.0,485713.0,1023282.0,537569.0,11013.72
1,17052889,0.055619,1.0,2.0,1465.0,6111.0,34449266.0,-119281531.0,12647.0,261.0,13091.0,2061.0,97099.0,1967.0,88000.0,464000.0,376000.0,5672.48
2,14186244,0.005383,2.0,3.0,1243.0,6059.0,33886168.0,-117823170.0,8432.0,261.0,21412.0,1286.0,97078.0,1962.0,85289.0,564778.0,479489.0,6488.30
3,12177905,-0.103410,3.0,4.0,2376.0,6037.0,34245180.0,-118240722.0,13038.0,261.0,396551.0,3101.0,96330.0,1970.0,108918.0,145143.0,36225.0,1777.51
4,12095076,-0.001011,3.0,4.0,2962.0,6037.0,34145202.0,-118179824.0,63000.0,261.0,47019.0,3101.0,96293.0,1950.0,276684.0,773303.0,496619.0,9516.26


In [4]:
# taking a look at the shape of the zillow data
df.shape

(52442, 18)

In [6]:
# looking at the columns in the zillow data
df.columns

Index(['parcelid', 'logerror', 'bathroomcnt', 'bedroomcnt',
       'calculatedfinishedsquarefeet', 'fips', 'latitude', 'longitude',
       'lotsizesquarefeet', 'propertylandusetypeid', 'regionidcity',
       'regionidcounty', 'regionidzip', 'yearbuilt',
       'structuretaxvaluedollarcnt', 'taxvaluedollarcnt',
       'landtaxvaluedollarcnt', 'taxamount'],
      dtype='object')

## Import Takeaways

- brought in zillow single unit properties from codeup database using my acquire.py file
- there are 18 columns and 52442 rows in the data
- now that we have the zillow data we can move into wrangle

# 3. Wrangle Zillow Data

In [7]:
# using my wrangle_zillow file and clean_zillow function on my zillow data
# set index to 'parcelid', handled missing values, dropped nulls, get latitude/longitude
df = wrangle_zillow.clean_zillow(df)

In [8]:
df.head() #check_yo_head

,logerror,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,propertylandusetypeid,regionidcity,regionidcounty,regionidzip,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount
parcelid,,,,,,,,,,,,,,,,,
14297519,0.025595,3.5,4.0,3100.0,6059.0,33.634931,-117.869207,4506.0,261.0,53571.0,1286.0,96978.0,1998.0,485713.0,1023282.0,537569.0,11013.72
17052889,0.055619,1.0,2.0,1465.0,6111.0,34.449266,-119.281531,12647.0,261.0,13091.0,2061.0,97099.0,1967.0,88000.0,464000.0,376000.0,5672.48
14186244,0.005383,2.0,3.0,1243.0,6059.0,33.886168,-117.823170,8432.0,261.0,21412.0,1286.0,97078.0,1962.0,85289.0,564778.0,479489.0,6488.30
12177905,-0.103410,3.0,4.0,2376.0,6037.0,34.245180,-118.240722,13038.0,261.0,396551.0,3101.0,96330.0,1970.0,108918.0,145143.0,36225.0,1777.51
12095076,-0.001011,3.0,4.0,2962.0,6037.0,34.145202,-118.179824,63000.0,261.0,47019.0,3101.0,96293.0,1950.0,276684.0,773303.0,496619.0,9516.26


In [9]:
# taking a look at the shape of the clean_zillow data
df.shape

(50913, 17)

In [10]:
# now using my prepare_zillow function from wrangle_zillow to add counties to the data
df = wrangle_zillow.prepare_zillow(df)

In [11]:
df.head() #check_yo_head

,logerror,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,propertylandusetypeid,regionidcity,regionidcounty,regionidzip,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,LA,Orange,Ventura
parcelid,,,,,,,,,,,,,,,,,,,,
14297519,0.025595,3.5,4.0,3100.0,6059.0,33.634931,-117.869207,4506.0,261.0,53571.0,1286.0,96978.0,1998.0,485713.0,1023282.0,537569.0,11013.72,0,1,0
17052889,0.055619,1.0,2.0,1465.0,6111.0,34.449266,-119.281531,12647.0,261.0,13091.0,2061.0,97099.0,1967.0,88000.0,464000.0,376000.0,5672.48,0,0,1
14186244,0.005383,2.0,3.0,1243.0,6059.0,33.886168,-117.823170,8432.0,261.0,21412.0,1286.0,97078.0,1962.0,85289.0,564778.0,479489.0,6488.30,0,1,0
12177905,-0.103410,3.0,4.0,2376.0,6037.0,34.245180,-118.240722,13038.0,261.0,396551.0,3101.0,96330.0,1970.0,108918.0,145143.0,36225.0,1777.51,1,0,0
12095076,-0.001011,3.0,4.0,2962.0,6037.0,34.145202,-118.179824,63000.0,261.0,47019.0,3101.0,96293.0,1950.0,276684.0,773303.0,496619.0,9516.26,1,0,0


In [12]:
# using create_features to help in explore
df = wrangle_zillow.create_features(df)

In [13]:
df.head() #check_yo_head

,logerror,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,propertylandusetypeid,regionidcity,...,taxrate,acres,acres_bin,sqft_bin,structure_dollar_per_sqft,structure_dollar_sqft_bin,land_dollar_per_sqft,lot_dollar_sqft_bin,bath_bed_ratio,cola
parcelid,,,,,,,,,,,,,,,,,,,,,
14297519,0.025595,3.5,4.0,3100.0,6059.0,33.634931,-117.869207,4506.0,261.0,53571.0,...,1.076313,0.103444,0.1,0.7,156.681613,0.5,119.300710,0.5,0.875000,0
17052889,0.055619,1.0,2.0,1465.0,6111.0,34.449266,-119.281531,12647.0,261.0,13091.0,...,1.222517,0.290335,0.3,0.3,60.068259,0.2,29.730371,0.3,0.500000,0
14186244,0.005383,2.0,3.0,1243.0,6059.0,33.886168,-117.823170,8432.0,261.0,21412.0,...,1.148823,0.193572,0.2,0.2,68.615447,0.2,56.865394,0.4,0.666667,0
12177905,-0.103410,3.0,4.0,2376.0,6037.0,34.245180,-118.240722,13038.0,261.0,396551.0,...,1.224661,0.299311,0.3,0.5,45.840909,0.1,2.778417,0.1,0.750000,0
12095076,-0.001011,3.0,4.0,2962.0,6037.0,34.145202,-118.179824,63000.0,261.0,47019.0,...,1.230599,1.446281,0.5,0.6,93.411209,0.3,7.882841,0.2,0.750000,0


In [14]:
# removed outliers for bathrooms, bedrooms, square feet, taxrate, and acres
df = wrangle_zillow.remove_outliers(df)

In [15]:
df.head() #check_yo_head

,logerror,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,propertylandusetypeid,regionidcity,...,taxrate,acres,acres_bin,sqft_bin,structure_dollar_per_sqft,structure_dollar_sqft_bin,land_dollar_per_sqft,lot_dollar_sqft_bin,bath_bed_ratio,cola
parcelid,,,,,,,,,,,,,,,,,,,,,
14297519,0.025595,3.5,4.0,3100.0,6059.0,33.634931,-117.869207,4506.0,261.0,53571.0,...,1.076313,0.103444,0.1,0.7,156.681613,0.5,119.300710,0.5,0.875000,0
17052889,0.055619,1.0,2.0,1465.0,6111.0,34.449266,-119.281531,12647.0,261.0,13091.0,...,1.222517,0.290335,0.3,0.3,60.068259,0.2,29.730371,0.3,0.500000,0
14186244,0.005383,2.0,3.0,1243.0,6059.0,33.886168,-117.823170,8432.0,261.0,21412.0,...,1.148823,0.193572,0.2,0.2,68.615447,0.2,56.865394,0.4,0.666667,0
12177905,-0.103410,3.0,4.0,2376.0,6037.0,34.245180,-118.240722,13038.0,261.0,396551.0,...,1.224661,0.299311,0.3,0.5,45.840909,0.1,2.778417,0.1,0.750000,0
12095076,-0.001011,3.0,4.0,2962.0,6037.0,34.145202,-118.179824,63000.0,261.0,47019.0,...,1.230599,1.446281,0.5,0.6,93.411209,0.3,7.882841,0.2,0.750000,0


In [17]:
# looking at the new shape of the wrangled zillow data
df.shape

(50677, 32)

In [18]:
# now I can split my data and move into explore
# using my_train_test_split(df) funtion
train, validate, test = wrangle_zillow.my_train_test_split(df)

train size:  908096
validate size:  389216
test size:  324352


## Wrangle Takeaways

- cleaned zillow data, set index to 'parcelid', removed nulls and missing values
- added latitude / longitude and counties to data
- added features and removed outliers on data
- split data into train, validate, and test

# 4. Data Exploration and Visuals